In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# Read prepped cohorts data

## Extract top results

- Read in prepared data and output

In [ ]:
results <- fread(paste0(SHARE_DIR, "1b_run_fishers_exact_with_interaction.csv")) 
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

- Collect top independent features

In [ ]:
results <- 
results %>% 
  mu( p_bh = p.adjust(fisher_pval, method = "BH")) %>% 
  fi(type != "univariate")

- Interation Event to Highlight

In [ ]:
highlighters <- 
c( "Pan-Cancer ## Alkaloid ## Platinum" = "cn_simple_chr11_q_gt75_or_purity_msIndelsPerMb_gt75", 
   "Prostate ## Anti-AR" = "chord_BRCA1_gt0_or_rna_geneset_KEGG_COMPLEMENT_AND_COAGULATION_CASCADES_gt25", 
   "Colon ## Chemotherapy" = "hotspot_KRAS_position_25398284_and_purity_tmbPerMb_gt50", 
   "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_OXIDATIVE_PHOSPHORYLATION_lt75_and_rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50",
   "Lung NSCLC ## Anti-PD-1" = "cn_simple_chr3_p_lt75_and_neo_ct_lt75", 
   "Breast Triple Negative ## Chemotherapy" = "cn_simple_chr22_q_gt25_and_rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50")

- 

In [ ]:
map <- 
c("e_nr" = "Event + No DCB", 
  "e_r" = "Event + DCB", 
  "ne_nr" = "No Event + No DCB", 
  "ne_r" = "No Event + DCB")

In [ ]:
tmp <- 
results %>% 
 se(cohortGo, feature, e_nr, ne_nr, e_r, ne_r, p_bh) %>% 
 ga(event, ct, -cohortGo, -feature, -p_bh) %>% 
 mu(event2 = factor(map[event], levels = unname(map)),  event = !m("No Event", event2), dcb = !m("No DCB", event2)) %>% 
 fi( cohortGo %in% names(highlighters) & feature %in% unlist(highlighters))

In [ ]:
alphas <- c("FALSE" = .4, "TRUE" = 1)
colors <- c("FALSE" = "grey", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
#tmp %>% ar(desc(ct))

In [ ]:
top <- 
tmp %>% 
 ggplot(aes(x = event2, y = ct, alpha = event, fill = dcb, color = event)) + 
 facet_wrap(~fct_reorder(cohortGo, p_bh, .fun = mean) + str_wrap(gsub("_", " ", gsub("rna_geneset_", "",feature)), width = 45) , ncol = 3, scales = "free") +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  scale_color_manual( values = colors) + 
  go_theme + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  geom_text(aes(x = 1.3, 
               y = 80, 
               label = paste0("p-value: ", formatC(p_bh, format = "e", digits = 1))), 
               vjust = -0.5, color = "black", size = 3) + 
  coord_cartesian(ylim = c(0,100)) + 
  labs(title = "Highlighted Combination Events with No Durable Benefit given Treatment", y = "Number of Patients\n", x = "Event and Durable Clinical Benefit") + 
  theme(strip.background = element_rect(fill = "white", color = "black"), legend.position = "none")

In [ ]:
options(repr.plot.width =12, repr.plot.height = 7)

In [ ]:
top

In [ ]:
ggsave( paste0(FIG_DIR, "Combination_events_with_no_durable_benefit.png"), plot = top, width = 12, height = 7)

# Generally Bad Prognostic Features 